# Task 1

In [118]:
import numpy as np


In [119]:
def Relu(k):
  
  for i in k:
    i = np.maximum(0,i)
  return k

In [120]:
def Softmax(k):
  exps = [np.exp(i) for i in k]
  summ = np.sum(exps)
  
  soft = np.array([i/summ for i in exps])
  # print(soft)
  return soft

In [121]:
def tanh(k):
  for i in k:
    # print("-------------")
    # print(i)
    
    i = (2/(1 + np.exp(-2*int(i))))-1
    # print(i)
    # print("-------------")
  return k

In [122]:
class Layer():

  def __init__(self, inputs, outputs, function):
    self.x,self.y = inputs
    self.n,self.k = outputs
    self.inputs = inputs
    self.outputs = outputs
    self.function = function
  
  def forward(self, matrix):
    matrix_w = np.random.uniform(-1,1,(self.y,self.k))
    result = np.dot(matrix,matrix_w)
    # print(result)
    return self.function(result)
  




In [123]:
class Model():
  
  def __init__(self):
    layer1 = Layer((1,256),(1,64),Relu)
    layer2 = Layer((1,64),(1,16), tanh)
    layer3 = Layer((1,16),(1,4), Softmax)
    
    self.layers = [layer1,layer2,layer3]


  def count(self, matrix):
    for layer in self.layers:
      matrix = layer.forward(matrix)
      
    
    return matrix

  def __call__(self, data):
        res = data
        for layer in self.layers:
            res = layer.forward(res)
        return res


In [124]:
model = Model()
X = np.random.uniform(-1,1, 256)

model.count(X)
# print(X.shape)

array([9.26361270e-08, 2.12364318e-35, 2.41337449e-18, 9.99999907e-01])

# Task 2

In [125]:
def MaxPool(matrix,l_in,l_out):
  res_ret = []
  step_stb = 0
  step_str = 0
  for flow in range(0,l_in):
      step_stb = 0
      step_str = 0
      mtx_res = []
      while (step_str < len(matrix)-1):
        k = 0
        mtx = matrix[:,:,flow]
        mat_res = []
        new_arr = []
        for i in range(step_stb,step_stb + 2):
          for j in range(step_str,step_str + 2):
            mat_res = np.append(mat_res,mtx[i][j])
        k = np.amax(mat_res)
        # print(k)
        # print("----")
        mtx_res = np.append(mtx_res,k)
        # print(mtx_res)
        # print("--------")
        
        step_stb+=2
        if step_stb >= len(matrix)-1:
          step_stb = 0
          step_str+=2

      mtx_res = mtx_res.reshape(int(len(matrix)/2),int(len(matrix)/2))
      # print(mtx_res)
      res_ret.append(mtx_res)
  # mat_res = mat_res.reshape(int(len(matrix)/2),int(len(matrix)/2)).astype(np.int32)
  return np.transpose(res_ret,axes=[1,2,0])



In [132]:
# x = np.random.uniform(-1,1,size=(4,4,3))
# # print(x)
# # print()
# res = MaxPool(x,3,3)
# print(np.shape(res))
# res

In [133]:
def sup_conv(matrix,layer_in,kernel):
  step_stb = 0
  step_str = 0
  lenght = (len(matrix)-len(kernel)) + 1
  new_matrix = []
  while (step_str < lenght):
    summ = 0
    for flow in range(0,layer_in):
      step_mtx = []
      ker = kernel[:,:,flow]
      mat = matrix[:,:,flow]
      for i in range(step_stb,step_stb + len(kernel)):
        for j in range(step_str,step_str + len(kernel)): 
          step_mtx = np.append(step_mtx,mat[i][j])
      step_mtx = step_mtx.reshape(len(kernel),len(kernel))
      mul_mtx = np.dot(ker,step_mtx)
      summ += np.sum(mul_mtx)
    new_matrix = np.append(new_matrix,summ) 
    step_stb += 1
    if step_stb >= lenght:
      step_stb = 0
      step_str+=1

  new_matrix = new_matrix.reshape(lenght,lenght)
  return new_matrix








In [134]:
def convolution(matrix,kernel, l_in, out):
  new_matrix = []
  for i in range(0, out):
    res = sup_conv(matrix,l_in,kernel)
    new_matrix.append(res)
  return np.transpose(new_matrix,axes=[1,2,0])


In [135]:
class Layer2():

  def __init__(self, inputs, outputs, function):
    self.x,self.y,self.z = inputs
    self.n,self.k,self.h = outputs
    self.inputs = inputs
    self.outputs = outputs
    self.function = function

  def forward(self, matrix):
    kernel = np.random.uniform(-1,1,size=(2,2,self.z))

    if self.function.__name__ == "MaxPool":
      return self.function(matrix,self.z,self.h)
    if self.function.__name__ == "convolution":
      return self.function(matrix,kernel,self.z,self.h)


In [139]:
class Model2():
  
  def __init__(self):
    layer1 = Layer2((19,19,3),(18,18,8),convolution)
    layer2 = Layer2((18,18,8),(9,9,8),MaxPool)
    layer3 = Layer2((9,9,8),(8,8,16),convolution)
    layer4 = Layer2((8,8,16),(4,4,16),MaxPool)
    
    self.layers = [layer1,layer2,layer3,layer4]


  def count(self, matrix):
    for layer in self.layers:
      print(np.shape(matrix))
      matrix = layer.forward(matrix)
    return matrix

  def __call__(self, data):
        res = data
        for layer in self.layers:
            res = layer.forward(res)
        return res

In [140]:
model = Model2()
X = np.random.uniform(-1,1,size=(19,19,3))
# kernel = np.random.randint(-1,1,size=(2,2,3))
res=model.count(X)

# print(res)
np.shape(res)

(19, 19, 3)
(18, 18, 8)
(9, 9, 8)
(8, 8, 16)


(4, 4, 16)

# Task 3

In [141]:
class Model3(Model,Model2):
    def __call__(self, data):
        model_1 = Model()
        model_2 = Model2()
        return model_1(np.ravel(model_2(data)))


In [150]:
X = np.random.uniform(-1,1 ,size=(19, 19, 3))
model_3 = Model3()
model_3(X)

<ipython-input-121-1e5581a792ce>:6: RuntimeWarning: overflow encountered in exp
  i = (2/(1 + np.exp(-2*int(i))))-1


array([6.02711401e-043, 8.33965386e-208, 1.00000000e+000, 1.20124164e-297])